# Modelado orientado a objetos a partir de la Solución Original en Clase
Se crearon 3 clases para la modelación:
* Solucion: Representa a una solucion, con metodos descatables neighbors, mutate and crossover.
* Flights: Guarda la informacion de los vuelos asi como metodos para interactuar con objetos tipo Solucion como una funcion costo, imprimir una representacion de la solucion, etc.
* Optimizador: Contiene ciertos proceso que tienen como objeto dar un objeto tipo Solucion minimizando una funcion costo dada.




In [1]:
import random
import time
import math

In [8]:
class Solucion:
    def __init__(self, solucion, domain = [(0,9)] * 12):
        self.solucion = solucion
        self.domain = domain

    def neighbors(self):
        '''
            regresa una lista con las soluciones que se forman al mover un vuelo uno arriba o uno abajo
        '''
        neighbors = []
        s = self.solucion
        for i in range(len(self.domain)):
            
            if s[i] > self.domain[i][0]:
                neighbors.append(Solucion(s[0:i] + [s[i] - 1] + s[i+1:]))
            if s[i] < self.domain[i][1]:
                neighbors.append(Solucion(s[0:i] + [s[i] + 1] + s[i+1:]))
        return neighbors

    def __str__(self):
        return str(self.solucion)

    def __repr__(self):
        return self.__str__()

    def __iter__(self):
        return self.solucion

    def __len__(self):
        return len(self.solucion)
        
    def __getitem__(self, indice):
        return self.solucion[indice]

    def mutate(self):
        '''
            Regresa un vecino 
        '''
        return random.choice(self.neighbors()) 
        
    def crossover(self, s2):
        '''
            Regreso una mezcla de ambas soluciones
        '''
        i = random.randint(1, len(self)-2)
        return Solucion(self[:i] + s2[i:])



In [173]:
class Flights:
    def __init__(self, path_flights, people, destination):
        self.destination = destination
        self.people = people
        self.flights = {}
        
        with open(path) as f:
            for line in f:
                origin, dest, t_depart, t_arrive, price = line.strip().split(',')
                self.flights.setdefault((origin, dest), [])
                self.flights[(origin, dest)].append((
                    self.get_minutes(t_depart), 
                    self.get_minutes(t_arrive), 
                    int(price)
                ))
    def print_sol(self, s):
        '''
            Representacion humana de la solucion
        '''
        assert isinstance(s, Solucion), 's debe ser un objeto tipo Solucion'
        for i in range(len(s)//2):
            
            dep = self.get_dep(i)
            persona = self.people[i][0]
            
            hora_dep1 = self.get_flights_dep(persona)[s[i*2]][0]
            hora_ar1 = self.get_flights_dep(persona)[s[i*2]][1]
            costo_dep1 = self.get_flights_dep(persona)[s[i*2]][2]
            
            hora_dep1 = self.get_hours(hora_dep1)
            hora_ar1 = self.get_hours(hora_ar1)
            costo_dep1 = '$' + str(costo_dep1).ljust(7, ' ')

            hora_dep2 = self.get_flights_ret(persona)[s[i*2 + 1]][0]
            hora_ar2 = self.get_flights_ret(persona)[s[i*2] + 1][1]
            costo_dep2 = self.get_flights_ret(persona)[s[i*2] + 1][2]
            
            hora_dep2 = self.get_hours(hora_dep2)
            hora_ar2 = self.get_hours(hora_ar2)
            costo_dep2 = '$' + str(costo_dep2).ljust(7, ' ')
            
        
            
            print(dep, 
                  persona.ljust(10, ' '), 
                  hora_dep1,
                  hora_ar1,
                  costo_dep1,
                  hora_dep2,
                  hora_ar2,
                  costo_dep2)
            
    
    def get_dep(self, nombre):
        if isinstance(nombre, str):
            for tupla in self.people:
                if tupla[0] == nombre:
                    return tupla[1]
        if isinstance(nombre, int):
            return self.people[nombre][1]
        

    def get_flights_dep(self, nombre):
        #asserts
        dep = self.get_dep(nombre)
        
        return self.flights[(dep,self.destination)]
            
    def get_flights_ret(self, nombre):
        #asserts
        dep = self.get_dep(nombre)
        return self.flights[(self.destination, dep)]
            
    def random_solucion(self):
        '''
            Regresa una solucion aleatoria
        '''
        solucion = []
        for tupla in self.people:
            nombre = tupla[0]
            solucion.append(
                random.choice(range(len(self.get_flights_dep(nombre))))
            )
            solucion.append(
                random.choice(range(len(self.get_flights_ret(nombre))))
            )
        return Solucion(solucion)

    def get_minutes(self, t):
        x = time.strptime(t, '%H:%M')
        h = x.tm_hour
        m = x.tm_min
        return 60 * h + m
    
    def get_hours(self, t):
        h = t // 60
        m = t - h * 60 
        h = str(h).rjust(2, '0')       
        m = str(m).rjust(2, '0')
        return h + ':' + m
        
        
    def schedule_cost(self, solucion):
        # contamos el precio total de cada vuelo (ida y regreso)
        total_price = 0
        s = solucion
        # nos interesa conocer el tiempo de llegada a NY mas tarde
        # y el tiempo de salida de NY mas temprano.
        latest_arrival = 0
        earliest_departure = 24 * 60
        
        for i in range(len(s) // 2):
            origin = self.people[i][1]
            out_flight = self.flights[(origin, self.destination)][s[2*i]]
            ret_flight = self.flights[(self.destination, origin)][s[2*i+1]]
            
            total_price += out_flight[2] # vuelo de ida
            total_price += ret_flight[2] # vuelo de regreso
            
            # tiempo de llegada máximo
            # tiempo de salida mínimo
            if latest_arrival < out_flight[1]:
                latest_arrival = out_flight[1]
            if earliest_departure > ret_flight[0]:
                earliest_departure = ret_flight[0]
        
        # contamos el tiempo de espera de cada persona
        total_wait = 0
        
        for i in range(len(s) // 2):
            origin = self.people[i][1]
            out_flight = self.flights[(origin, self.destination)][s[2*i]]
            ret_flight = self.flights[(self.destination, origin)][s[2*i+1]]
            
            # todos esperan al último familiar en llegar
            total_wait += latest_arrival - out_flight[1]
            
            # todos llegan al aeropuerto al mismo tiempo y esperan su vuelo
            total_wait += ret_flight[0] - earliest_departure
            
            # si el último en llegar a NY llega después del primero en
            # irse de NY se paga un día más de la renta del carro.
            # el costo de la renta por un día es independiente de la
            # solución.
            if latest_arrival > earliest_departure:
                total_price += 50
        
        # El costo total es el precio total de los vuelos y el tiempo de
        # espera total de las personas.
        # Buscamos soluciones con un bajo costo.
        return total_price + total_wait


    




        
    

In [120]:
class Optimizador:
    def __init__(self, flights, fun_cost = None):
        self.flights = flights
        
        if not fun_cost:
            self.fun_cost = self.flights.schedule_cost
        else:
            self.fun_cost = fun_cost 
        
        
    def solve_randomly(self, repeats = 1000):
        '''
            Devuelvela mejor solucion encontrada de manera aleatorio
        '''
        best_cost = float('inf')
        best_sol = None
        
        for _ in range(repeats):
            s = self.flights.random_solucion()
            c = self.fun_cost(s)
            if c < best_cost:
                best_cost = c
                best_sol = s
        
        return best_sol

    def solve_hillclimbing(self, repeats = 1000, sol_ini = None):
        '''
            Devuelvela mejor solucion encontrada de con hill climbing
        '''
        if  sol_ini is None:
            s = self.flights.random_solucion()
        else:
            s = sol_ini
        contador = 0
        while (True and contador <= 1000):
            contador = contador + 1
            neighbors = s.neighbors()
            cost = self.fun_cost(s)
            best_neighbor = min(neighbors, key=self.fun_cost)
            neighbor_cost = self.fun_cost(best_neighbor)
            
            if cost <= neighbor_cost:
                return s
            
            s = best_neighbor
    def solve_annealing(self, Ti=10000.0, Tf=0.1, alpha=0.95, sol_ini = None):
        '''
            Devuelvela mejor solucion encontrada con annealing
        '''
        cost_of = self.fun_cost
        if  sol_ini is None:
            solution = self.flights.random_solucion()
        else:
            solution = sol_ini
            
        cost = cost_of(solution)
        T = Ti
        while T > Tf:
            neighbor = random.choice(solution.neighbors())
            neighbor_cost = cost_of(neighbor)
            diff = cost - neighbor_cost
            if diff > 0 or random.random() < math.exp(diff / T):
                solution = neighbor
                cost = neighbor_cost
            T = alpha*T
        
        return solution

    def solve_evolving(self, pop_size=50, mut_prob=0.2, elite=0.2, epochs=100, sol_ini = None):
        '''
            Devuelvela mejor solucion encontrada con evolving
        '''
        if  sol_ini is None:
            pop = [self.flights.random_solucion() for _ in range(pop_size)]
        else:
            pop = [self.flights.random_solucion() for _ in range(pop_size-1) ]
            pop.append(sol_ini)
        
        
        top_elite = int(elite * pop_size)
        
        for epoch in range(epochs):
            pop.sort(key=self.fun_cost)
            
            best = pop[:top_elite]
            while len(best) < pop_size:
                if random.random() < mut_prob:
                    best.append(
                        best[random.randint(0, top_elite-1)].mutate()
                    )
                else:
                    #aveces se keda el mismo
                    best.append(
                        best[random.randint(0, top_elite-1)].crossover(
                        best[random.randint(0, top_elite-1)])
                    )
            pop = best
        pop.sort(key=self.fun_cost)
        return pop[0]
        


    

## Implementacion

In [121]:
path = 'assets/schedule.txt'
people = [('Seymour', 'BOS'),
          ('Franny', 'DAL'),
          ('Zooey', 'CAK'),
          ('Walt', 'MIA'),
          ('Buddy', 'ORD'),
          ('Les', 'OMA')]
destination = 'LGA'

In [174]:
#creamos objetos
vuelos = Flights(path, people, destination)
optimizador = Optimizador(yo, fun_cost= yo.schedule_cost)

#probamos los 4 distintos metodos para optimizar una solucion
s1 = optimizador.solve_annealing()
s2 = optimizador.solve_hillclimbing()
s3 = optimizador.solve_evolving()
s4 = optimizador.solve_randomly()

#vemos cual es el mejor de los 4
S = ((s1, 'annealing'),(s2, 'hillclimbing'),(s3, 'evolving'),(s4, 'random'))
for s, metodo in S:
    print(metodo + ' : ' + str(vuelos.schedule_cost(s)))

annealing : 4036
hillclimbing : 3263
evolving : 3125
random : 4202


In [175]:
vuelos.print_sol(s3)

BOS Seymour    13:40 15:37 $138     06:39 16:58 $62     
DAL Franny     10:30 14:57 $290     09:49 16:34 $500    
CAK Zooey      13:40 15:38 $137     08:19 18:45 $243    
MIA Walt       11:28 14:40 $248     06:33 15:05 $170    
ORD Buddy      14:22 16:32 $126     07:50 17:23 $189    
OMA Les        15:03 16:42 $135     08:04 18:56 $144    


## Funcion costo propuesta
La funcion costo que propongo toma en cuenta lso tiempos de espera maximos en los aeropuertos, tiempo de convivencia y costo total de los vuelos.

In [176]:
def fun_costo_c(flights):
    '''
        Regresa una funcion costo para las soluciones basadas en el tiempo de convencia maxima, tiempos de
        espera maximos y precio
    '''
    def fun_cost(solucion):
        total_price = 0
        s = solucion

        time_arrivals = []
        time_departures = []
        
        for i in range(len(s) // 2):
            origin = flights.people[i][1]
            out_flight = flights.flights[(origin, flights.destination)][s[2*i]]
            ret_flight = flights.flights[(flights.destination, origin)][s[2*i+1]]
            
            total_price += out_flight[2] # vuelo de ida
            total_price += ret_flight[2] # vuelo de regreso
            
            #agregamos tiempos a listas
            time_arrivals.append(out_flight[1])
            time_departures.append(ret_flight[0])

        t_convivencia = min(time_departures) - max(time_arrivals) 

        t_espera_max_ar = max(time_arrivals) - min(time_arrivals)

        t_espera_max_re = max(time_departures) - min(time_departures)

        costo = 0.5 * t_convivencia + 0.2 * (t_espera_max_ar + t_espera_max_re) + 0.3 * total_price

        if t_convivencia < 0:
            return costo * 100000000
        else:
            return costo
            
    return fun_cost
        


In [177]:
#probamos funcion
#con una solucion que si tenga tiempo de convivencia positivo
s_razonable = Solucion([*(4, 6) * 6])

fun_costo_c(vuelos)(s_razonable)

76259999999.99998

In [170]:
#corremos optimizador de nuevo
optimizador2 = Optimizador(vuelos, fun_cost = fun_costo_c(vuelos))

#empezamos con la solucion de arriba 
s11 = optimizador2.solve_annealing(sol_ini = s_razonable)
s22 = optimizador2.solve_hillclimbing(sol_ini = s_razonable)
s33 = optimizador2.solve_evolving(pop_size = 1000, elite = 0.01, sol_ini = s_razonable)
s44 = optimizador2.solve_randomly()

#vemos cual es el mejor de los 4
S = ((s11, 'annealing'),(s22, 'hillclimbing'),(s33, 'evolving'),(s44, 'random'))
for s, metodo in S:
    print(metodo + ' : ' + str(optimizador2.fun_cost(s)))

#lo corri varias veces para obtener un numero bajo (t_convivencia > 0)

annealing : 54850000000.0
hillclimbing : 54850000000.0
evolving : 695.8
random : 37900000000.0


In [178]:
vuelos.print_sol(s33)

BOS Seymour    12:34 15:02 $109     15:25 15:30 $74     
DAL Franny     06:12 10:22 $230     17:14 11:15 $347    
CAK Zooey      08:27 10:45 $139     15:50 12:56 $249    
MIA Walt       09:15 12:29 $225     20:27 14:38 $262    
ORD Buddy      12:44 14:17 $134     17:06 17:09 $190    
OMA Les        09:15 12:03 $99      15:07 13:24 $171    
